In [140]:
import pandas as pd
from ipywidgets import widgets
from IPython.display import display
pd.options.display.max_colwidth = 10000
pd.options.display.max_rows = 10000

all_payments = pd.read_csv("/Users/johnclark/Documents/Finances/All_Payment_Methods020122.csv")
end_of_year = pd.read_csv("/Users/johnclark/Documents/Finances/Year End All Payment Methods020122.csv",parse_dates=[2], header=0)
end_of_year["Amount_value"] = end_of_year["Amount"].apply(lambda x: float(x[1:].replace(",","")) if x.startswith("$") else -float(x[2:].replace(",","")) if x.startswith("-$") else x)

no_credit_card = end_of_year[end_of_year["Payee"] != "WELLS FARGO CREDIT CARD"]
no_wells_credit_rec_trans = end_of_year[(end_of_year["Payee"] != "WELLS FARGO CREDIT CARD") & (~end_of_year["Description"].str.startswith("RECURRING TRANSFER")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J SAVINGS XXXXXX5751")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9873"))]

year = 2021
utility_check = pd.read_csv("/Users/johnclark/Documents/Finances/Utility{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
utility_check["account_name"] = "utility"
umbrella_check = pd.read_csv("/Users/johnclark/Documents/Finances/Umbrella{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
umbrella_check["account_name"] = "umbrella"
savings_check = pd.read_csv("/Users/johnclark/Documents/Finances/Savings{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
savings_check["account_name"] = "savings"
primary_check = pd.read_csv("/Users/johnclark/Documents/Finances/Primary{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
primary_check["account_name"] = "primary"
# platinum_card = pd.read_csv("/Users/johnclark/Documents/Finances/PlatinumCard{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
# platinum_card["account_name"] = "platinum"
# cashwise_card = pd.read_csv("/Users/johnclark/Documents/Finances/CashWiseCard{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
# cashwise_card["account_name"] = "cashwise"

accounts_joined = utility_check.append(umbrella_check)
accounts_joined = accounts_joined.append(savings_check)
accounts_joined = accounts_joined.append(primary_check)
# accounts_joined = accounts_joined.append(platinum_card)
# accounts_joined = accounts_joined.append(cashwise_card)

accounts_joined.reset_index(inplace=True, drop=True)

assert len(accounts_joined) == len(utility_check) + len(umbrella_check) + len(savings_check) + len(primary_check)# + len(platinum_card) + len(cashwise_card)

accounts_joined["spending"] = accounts_joined["Amount"].apply(lambda x: x if x<0.0 else 0.0)
accounts_joined["income"] = accounts_joined["Amount"].apply(lambda x: x if x>0.0 else 0.0)

exclude_desc = {"FID BKG SVC LLC MONEYLINE 210402 X49800767 O0TLA JOHN P CLARK","RECURRING TRANSFER TO CLARK",
                "ONLINE TRANSFER TO CLARK J CHECKING XXXXXX7135", 
                "ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9865", 
                "ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9873", 
                "ONLINE TRANSFER TO CLARK J SAVINGS XXXXXX5751", 
                "ONLINE TRANSFER FROM CLARK J CHECKING XXXXXX7135",
                "ONLINE TRANSFER FROM CLARK J CHECKING XXXXXX9865",
                "ONLINE TRANSFER FROM CLARK J CHECKING XXXXXX9873",
                "ONLINE TRANSFER FROM CLARK J SAVINGS XXXXXX5751",
                "ONLINE TRANSFER FROM CLARK J",
                "ONLINE TRANSFER TO CLARK J"}
tithe_desc_starts = {"RURAL INDIA MINI", "BILL PAY Church of the Open Door",
                     "CHECK # 136","BUSINESS TO BUSINESS ACH www.gracepres.co", 
                     "CHECK # 198",
                     "BILL PAY Church of the Resurrection",
                     "Flagstaff Christ",
                     "Mount Calvary"}
tax_desc_starts = {"CHECK # 132",
                   "CHECK # 131",
                   "FRANCHISE TAX BD CASTTAXRFD"}
shoffice_starts = {"CHECK # 129","DEPOSITED OR CASHED CHECK # 128","CASHED CHECK # 127"}

accounts_joined["chase"] = accounts_joined.apply(lambda r: r["Amount"] if r["Description"].startswith("CHASE") else 0.0, raw=False, axis=1)
accounts_joined["tithe"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tithe_desc_starts if r["Description"].startswith(td)]) > 0  else 0.0, raw=False, axis=1)
accounts_joined["taxes"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tax_desc_starts if r["Description"].startswith(td)]) > 0  else 0.0, raw=False, axis=1)
accounts_joined["taxes_spend"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tax_desc_starts if r["Description"].startswith(td)]) > 0 and r["Amount"] < 0.0  else 0.0, raw=False, axis=1)
accounts_joined["shoffice"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in shoffice_starts if r["Description"].startswith(td)]) > 0 and r["Amount"] < 0.0  else 0.0, raw=False, axis=1)


accounts_joined["filtered_amount"] = accounts_joined.apply(lambda r: r["Amount"] if not r["Description"] in exclude_desc else 0.0, raw=False, axis=1)
accounts_joined["filtered_income"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]>0.0 and not r["Description"] in exclude_desc else 0.0, raw=False, axis=1)

accounts_joined["filtered_spending"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]<0.0 and len([1 for td in exclude_desc if r["Description"].startswith(td)]) == 0 else 0.0, raw=False, axis=1)
accounts_joined["filtered_spending"] = accounts_joined["filtered_spending"] - accounts_joined["tithe"] - accounts_joined["taxes_spend"] - accounts_joined["shoffice"]

accounts_joined["comment"] = ""

notes = {"CHECK # 132":"Arizona Department of Revenue",
         "CHECK # 131":"United States Treasury",
         "CHECK # 129":"Master Mechanical",
         "CHECK # 130":"Hunting rifle",
         "CHECK # 126":"Evelyn enrollment",
         "CASHED CHECK # 127":"Dan shoffice reno",
         "DEPOSITED OR CASHED CHECK # 128":"Shoffice electrical",
         "FID BKG SVC LLC MONEYLINE 210402 X49800767 O0TLA JOHN P CLARK":"Transfer from fidelity for tithe",
         "FRANCHISE TAX BD CASTTAXRFD 040521 XXXXX8570 CLARK III, JOHN P & HA":"Franchise tax board CASTAX",
         "CHASE CREDIT CRD EPAY 210223 5147608953 JOHN CLARK III":"2365.00 to ACSTO",
         "CHASE CREDIT CRD EPAY 210824 5467211881 JOHN CLARK III":"600 FCS, 500.38 watermans CA, 653.99 apple, 2619.25 laptop",
         "CHASE CREDIT CRD EPAY 211022 5572610997 JOHN CLARK III":"4515.75 FCS",
         "CHASE CREDIT CRD EPAY 211226 5690175783 JOHN CLARK III":"869.16 supressor, 2117.98 chair, 2435 ATC"}
for idx, row in accounts_joined.iterrows():
    if row["Description"] in notes:
        note = notes[row["Description"]]
        accounts_joined.loc[idx,"comment"] = note



In [141]:
import numpy as np

columns = ["Amount","income","filtered_amount","taxes","filtered_income", "spending","filtered_spending","chase","tithe","taxes_spend","shoffice"]
sum_table = accounts_joined.groupby(pd.Grouper(key='Date', freq='1M')).sum()[columns]
sum_table.loc["Totals"] = sum_table[columns].sum()

def red(s, props=''):
    return np.where(s < 0.0, props, '')
slice_1 = ["filtered_spending"]
slice_2 = ["chase"]
slice_3 = ["tithe","taxes_spend","shoffice"]
df = sum_table.style.apply(red, props='color:red;', axis=0).set_properties(**{'background-color': '#ffffb3'}, subset=slice_1).set_properties(**{'background-color': '#e8e8e8'}, subset=slice_2).set_properties(**{'background-color': '#52ffff'}, subset=slice_3)
#df = df.style.apply(red, props='color:red;', axis=0, subset=slice_2).set_properties(**{'background-color': '#3E85BD'}, subset=slice_2)
display(df)
#display(sum_table)





,Amount,income,filtered_amount,taxes,filtered_income,spending,filtered_spending,chase,tithe,taxes_spend,shoffice
Date,,,,,,,,,,,
2021-01-31 00:00:00,-3262.540000,12768.210000,-3262.540000,0.000000,12768.210000,-16030.750000,-8728.780000,-5536.140000,-2800.250000,0.000000,0.000000
2021-02-28 00:00:00,-9186.000000,12893.100000,-9186.000000,0.000000,12893.100000,-22079.100000,-12485.130000,-7927.580000,-2800.250000,0.000000,-2192.000000
2021-03-31 00:00:00,-6911.470000,32993.120000,-6911.470000,0.000000,32993.120000,-39904.590000,-10527.620000,-5281.050000,-12800.250000,0.000000,-1875.000000
2021-04-30 00:00:00,66655.410000,102292.200000,16655.410000,2469.000000,52292.200000,-35636.790000,-7311.820000,-4580.390000,-2800.250000,-19923.000000,0.000000
2021-05-31 00:00:00,304.590000,13559.350000,304.590000,0.000000,13559.350000,-13254.760000,-5952.790000,-3412.340000,-2800.250000,0.000000,0.000000
2021-06-30 00:00:00,-1369.970000,14327.580000,-1369.970000,0.000000,14327.580000,-15697.550000,-8195.580000,-4841.890000,-2800.250000,0.000000,0.000000
2021-07-31 00:00:00,-6443.260000,60652.370000,-6443.260000,0.000000,60652.370000,-67095.630000,-9793.660000,-6998.290000,-7800.250000,0.000000,0.000000
2021-08-31 00:00:00,-5549.340000,13760.760000,-5549.340000,0.000000,13760.760000,-19310.100000,-11808.130000,-8118.730000,-2800.250000,0.000000,0.000000
2021-09-30 00:00:00,-8070.010000,13661.810000,-8070.010000,0.000000,13661.810000,-21731.820000,-9329.850000,-5999.850000,-7800.250000,0.000000,0.000000


## Show month

In [143]:

month = 10

start_date = '2021-{}-01'.format(month)
if month < 12:
    end_date = '2021-{}-01'.format(month+1)
else:
    end_date = '2022-01-01'
view_table = accounts_joined[(accounts_joined["Date"] >= start_date) & (accounts_joined["Date"] < end_date)].sort_values("Date")

view_table



,Date,Amount,Unk,CheckNum,Description,account_name,spending,income,chase,tithe,taxes,taxes_spend,shoffice,filtered_amount,filtered_income,filtered_spending,comment
151,2021-10-01,400.00,*,NaN,RECURRING TRANSFER FROM CLARK J REF #OP0CJHHD5V SAVINGS TO COVER GAS ELECTRIC WATER IN RENTAL,umbrella,0.00,400.00,0.00,0.00,0.0,0.0,0.0,400.00,400.00,0.00,
197,2021-10-01,-400.00,*,NaN,RECURRING TRANSFER TO CLARK J REF #OP0CJHHD5V PORTFOLIO CHECKING TO COVER GAS ELECTRIC WATER IN RENTAL,savings,-400.00,0.00,0.00,0.00,0.0,0.0,0.0,-400.00,0.00,0.00,
276,2021-10-01,-2091.47,*,NaN,RECURRING TRANSFER TO CLARK J REF #OP0CJG8MJ9 CHECKING MORTGAGE PAYMENT,primary,-2091.47,0.00,0.00,0.00,0.0,0.0,0.0,-2091.47,0.00,0.00,
31,2021-10-01,2091.47,*,NaN,RECURRING TRANSFER FROM CLARK J REF #OP0CJG8MJ9 CHECKING MORTGAGE PAYMENT,utility,0.00,2091.47,0.00,0.00,0.0,0.0,0.0,2091.47,2091.47,0.00,
30,2021-10-01,-2050.88,*,NaN,DOVENMUEHLE MTG MORTG PYMT 100121 1465323044 John P Clark III,utility,-2050.88,0.00,0.00,0.00,0.0,0.0,0.0,-2050.88,0.00,-2050.88,
277,2021-10-01,4529.44,*,NaN,APPLE INC. PAYROLL 489652 John P Clark III,primary,0.00,4529.44,0.00,0.00,0.0,0.0,0.0,4529.44,4529.44,0.00,
28,2021-10-04,-800.00,*,NaN,"RURAL INDIA MINI CONTRIBUTN 211004 XX014IR691L5NW CLARK, JOHN",utility,-800.00,0.00,0.00,-800.00,0.0,0.0,0.0,-800.00,0.00,0.00,
29,2021-10-04,800.00,*,NaN,RECURRING TRANSFER FROM CLARK J REF #OP0CK7Y82H CHECKING TRANFER FOR RIM TITHE,utility,0.00,800.00,0.00,0.00,0.0,0.0,0.0,800.00,800.00,0.00,
275,2021-10-04,-800.00,*,NaN,RECURRING TRANSFER TO CLARK J REF #OP0CK7Y82H CHECKING TRANFER FOR RIM TITHE,primary,-800.00,0.00,0.00,0.00,0.0,0.0,0.0,-800.00,0.00,0.00,
27,2021-10-12,600.25,*,NaN,RECURRING TRANSFER FROM CLARK J REF #OP0CLLXK9K CHECKING GRACE GIVING,utility,0.00,600.25,0.00,0.00,0.0,0.0,0.0,600.25,600.25,0.00,


In [ ]:
eoy_primary_spend = end_of_year[end_of_year["Payment Method"].str.startswith("Primary ck acct")]
primary_spend = primary_check[primary_check["Amount"] < 0]

display(eoy_primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount_value"))
display(primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount"))



In [4]:
view_table["Description"]

-39904.59

In [113]:
1182.10+46.25+231.14+61.71-80.44

1440.7599999999998